In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# --- 我们先重新加载和切分数据，确保所有步骤都在一个代码块里 ---
file_path = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
train_full = pd.read_csv(file_path)

X_full = train_full.drop('SalePrice', axis=1)
y_full = train_full['SalePrice']
X_train, X_val, y_train, y_val = train_test_split(X_full, y_full, test_size=0.2, random_state=42)

# 对目标变量y进行对数变换
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)
# --------------------------------------------------------------------


# --- 核心代码开始 ---

# 1. 再次区分数值特征和类别特征 (基于X_train)
#    我们从列中移除'Id'，因为它不是一个有用的特征
X_train = X_train.drop('Id', axis=1)
X_val = X_val.drop('Id', axis=1)

categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]


# 2. 创建针对不同类型特征的预处理"管道"

# 为数值型特征创建一个管道：
# 步骤1: 用中位数(median)填充缺失值
numerical_transformer = SimpleImputer(strategy='median')

# 为类别型特征创建一个管道：
# 步骤1: 用众数(most_frequent)填充缺失值
# 步骤2: 进行独热编码(OneHotEncoder)，忽略未知类别
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 3. 将这两个管道用 ColumnTransformer 组合成一个总的预处理器
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


# 4. 用预处理器来处理数据
# 注意：预处理器在训练集上用 .fit_transform() (学习并转换)
X_train_processed = preprocessor.fit_transform(X_train)

# 在验证集上只用 .transform() (只用从训练集学到的规则去转换)
X_val_processed = preprocessor.transform(X_val)


# 5. 查看处理后的数据形状
print("原始训练集特征形状:", X_train.shape)
print("处理后训练集特征形状:", X_train_processed.shape)
print("\n原始验证集特征形状:", X_val.shape)
print("处理后验证集特征形状:", X_val_processed.shape)
print("\n✅ 数据预处理完成！")

原始训练集特征形状: (1168, 79)
处理后训练集特征形状: (1168, 285)

原始验证集特征形状: (292, 79)
处理后验证集特征形状: (292, 285)

✅ 数据预处理完成！


In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
import numpy as np

# 1. 定义并训练模型
# 我们选择随机森林回归器作为基准模型，设置 random_state 以保证结果可复现
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

print("模型训练中...")
# 用处理好的训练数据和对数变换后的目标值进行训练
model.fit(X_train_processed, y_train_log)
print("✅ 模型训练完成！")

# 2. 在验证集上进行预测
# 模型预测出的也是对数变换后的值
val_preds_log = model.predict(X_val_processed)

# 3. 将预测值逆变换回原始尺度
# np.expm1() 是 np.log1p() 的逆运算
val_preds = np.expm1(val_preds_log)

# 4. 计算并打印评估分数
# Kaggle 使用的是均方根对数误差 (RMSLE)
# 我们用 mean_squared_log_error 计算出 MSLE，然后开方得到 RMSLE
# 注意：y_val 是原始的、未经对数变换的验证集目标值
rmsle = np.sqrt(mean_squared_log_error(y_val, val_preds))

print("-" * 50)
print(f"🎉 恭喜！你的基准模型的验证集RMSLE分数为: {rmsle:.5f}")
print("-" * 50)

模型训练中...
✅ 模型训练完成！
--------------------------------------------------
🎉 恭喜！你的基准模型的验证集RMSLE分数为: 0.14561
--------------------------------------------------


In [6]:
# 假设 preprocessor 和 model 变量已经存在于之前的代码块中

# 1. 加载官方测试数据
test_data_path = '/kaggle/input/house-prices-advanced-regression-techniques/test.csv'
test_df = pd.read_csv(test_data_path)

# 2. 准备测试数据 (保存Id，并去除Id列)
test_ids = test_df['Id']
test_X = test_df.drop('Id', axis=1)

# 3. 使用【已经拟合过】的预处理器来转换测试数据
# ‼️注意：这里只用 .transform()，绝不能用 .fit_transform()
test_X_processed = preprocessor.transform(test_X)

# 4. 使用【已经训练好】的模型进行预测
test_preds_log = model.predict(test_X_processed)

# 5. 将预测结果逆变换回原始尺度
test_preds = np.expm1(test_preds_log)

# 6. 创建提交文件
submission_df = pd.DataFrame({'Id': test_ids, 'SalePrice': test_preds})

# 7. 保存为 .csv 文件
submission_df.to_csv('submission_baseline.csv', index=False)

print("-" * 50)
print("🎉 提交文件 'submission_baseline.csv' 已成功生成！")
print("现在你可以去Kaggle的 'Output' 部分找到它并提交了。")
print("-" * 50)

--------------------------------------------------
🎉 提交文件 'submission_baseline.csv' 已成功生成！
现在你可以去Kaggle的 'Output' 部分找到它并提交了。
--------------------------------------------------
